In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=9a832411e375e1be095d276da1fb583208929f784872a6a41d6310b2aede9f6e
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 3.3MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F20 '''
def F20(X):
    f = bn.F20()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 30.57 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_204000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_20_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(10, 20):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-10,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 10
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=391461, Sat Apr 18 19:36:32 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.092842618178097e+06 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.919520256003844e+06 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [2210283.01498147] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [2123437.50449948] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.846961987236538e+06 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   27    513 4.021411817105741e+05 1.1e+00 2.10e+00  2e+00  2e+00 0:03.4
   59   1121 1.633195591159656e+05 1.1e+00 1.89e+00  2e+00  2e+00 0:07.4
  100   1900 -4.975018447273667e+05 1.2e+00 1.35e+00  1e+00  1e+00 0:12.5
  149   2831 -1.335634982024415e+06 1.2e+00 8.49e-01  8e-01  9e-01 0:18.5
  200   3800 -1.911275220138605e+06 1.2e+00 4.97e-01  5e-01  5e-01 0:24.9
  265   5035 -2.242755284776406e+06 1.2e+00 2.38e-01  2e-01  2e-01 0:33.0
  300   5700 -2.289812227838916e+06 1.2e+00 1.62e-01  2e-01  2e-01 0:37.4
  381   7239 -2.328514329038660e+06 1.2e+00 6.61e-02  7e-02  7e-02 0:47.4
  400   7600 -2.331323286177823e+06 1.2e+00 5.23e-02  5e-02  5e-02 0:49.7
  497   9443 -2.336088796298889e+06 1.2e+00 1.75e-02  2e-02  2e-02 1:01.7
  500   9500 -2.336157928964107e+06 1.2e+00 1.69e-02  2e-02  2e-02 1:02.1
  600  11400 -2.336700485452680e+06 1.2e+00 5.86e-03  6e-03  6e-03 1:14.8
  700  13300 -2.336758551530716e+06 1.2e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [1969341.12838215] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [2348429.69815666] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.995984746166652e+06 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.745192590467214e+06 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


...

    3     57 1.244351434518659e+06 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   30    570 3.459690275466095e+05 1.1e+00 2.02e+00  2e+00  2e+00 0:03.4
   65   1235 1.828825001610024e+04 1.2e+00 1.81e+00  2e+00  2e+00 0:07.5
  100   1900 -5.423047199491379e+05 1.2e+00 1.41e+00  1e+00  1e+00 0:11.5
  153   2907 -1.314590468211066e+06 1.2e+00 8.38e-01  8e-01  8e-01 0:17.5
  200   3800 -1.882978395324411e+06 1.2e+00 5.08e-01  5e-01  5e-01 0:23.0
  269   5111 -2.238445558435519e+06 1.2e+00 2.29e-01  2e-01  2e-01 0:31.1
  300   5700 -2.288854841661646e+06 1.2e+00 1.61e-01  2e-01  2e-01 0:34.6
  388   7372 -2.329696937593304e+06 1.2e+00 6.30e-02  6e-02  6e-02 0:44.6
  400   7600 -2.330945846605473e+06 1.2e+00 5.54e-02  5e-02  6e-02 0:46.0
  500   9500 -2.336134241992240e+06 1.2e+00 1.84e-02  2e-02  2e-02 0:57.3
  600  11400 -2.336705579516165e+06 1.2e+00 5.98e-03  6e-03  6e-03 1:08.7
  700  13300 -2.336758443729247e+06 1.3e+00 2.01e-03  2e-03  2e-03 1:20.0
  800  15200 -2.336765527660920e+06 1.3e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [1969341.12838215] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [1433827.19892451] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.370832375800564e+06 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.264133718159482e+06 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [1871092.08373049] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   34    646 4.270709606945949e+05 1.1e+00 1.96e+00  2e+00  2e+00 0:03.3
   76   1444 4.116392046256688e+05 1.2e+00 1.78e+00  2e+00  2e+00 0:07.4
  100   1900 3.070502119169023e+05 1.2e+00 1.55e+00  2e+00  2e+00 0:09.7
  162   3078 8.921084514690470e+04 1.2e+00 1.09e+00  1e+00  1e+00 0:15.8
  200   3800 -7.327226128818572e+04 1.2e+00 8.39e-01  8e-01  9e-01 0:19.5
  283   5377 -2.816566556869993e+05 1.2e+00 4.80e-01  5e-01  5e-01 0:27.5
  300   5700 -3.213264543298322e+05 1.2e+00 4.37e-01  4e-01  4e-01 0:29.1
  400   7600 -3.841751950898077e+05 1.2e+00 2.24e-01  2e-01  2e-01 0:38.8
  500   9500 -4.081626226743101e+05 1.2e+00 1.14e-01  1e-01  1e-01 0:48.5
  600  11400 -4.172973224277610e+05 1.3e+00 7.29e-02  7e-02  7e-02 0:58.3
  700  13300 -4.230483188832179e+05 1.5e+00 6.09e-02  6e-02  6e-02 1:08.0
  800  15200 -4.267650722123090e+05 1.9e+00 5.61e-02  5e-02  7e-02 1:17.7
  900  17100 -4.285405386969731e+05 2.1e+00 3.90e-02  4e-02  5e-02 1:27.4
 1000  19000 -4.296529854048053e+05 2.2e+0

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('sample_data\\F20_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F20(X_Values.iloc[i,:200])
    SVM_Fun [i] = F20(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F20(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([208029.50531966]), count=array([1])) 0.002935326701734779
SVM
ModeResult(mode=array([1494511.7831825]), count=array([1])) 268456.87344168365
ELN
ModeResult(mode=array([70361.46521547]), count=array([1])) 6678.864533078506
